In [2]:
import findspark                                              #Import library to Search for Spark Installation  

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          # Import of Spark Session
from pyspark import SparkContext as spark                     # Import the Regular Spark Contex 
from pyspark.sql import SQLContext                            # Import the SQL Spark Contex 
from pyspark.sql.window import Window                         # Import Window methoed
from pyspark.sql.functions import *                           # Import all Functions
from pyspark.sql.types import *                               # Import all Types
spark = SparkSession.builder.getOrCreate()                    # Creating Session
from datetime import date                                     # Import date methoed

import os
import pandas as pd
import boto3
import time
from botocore.client import ClientError

import pyarrow.parquet as pq
import s3fs
import calendar

month_ref = date.today().month                                # Creating Current Month Variable
year_ref = date.today().year                                  # Creating Current Year Variable

sc = spark.sparkContext                                       #Initialize Spark 

spark.conf.set("spark.sql.broadcastTimeout",  36000)          #Changing Broadcast Timeout
#conf = pyspark.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
#conf.set("spark.executor.heartbeatInterval","3600s")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1575630339505_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---

# Creating Variables

## Databases

In [ ]:
ATHENA_SPG = os.environ['SPG_DATABASE']

In [ ]:
ATHENA_HANA = os.environ['GERDAU_HANA_DATABASE']

In [ ]:
ATHENA_SALES = os.environ['GERDAU_SALES_DATABASE']

## Buckets

In [ ]:
SPG_MANUAL_INPUT_BUCKET = os.environ['MANUAL_INPUT_BUCKET']

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

In [ ]:
SPG_INPUT_BUCKET = os.environ['INPUT_BUCKET_FROM_OUTPUT']

## Input Paths

In [ ]:
# Wallet (Sales Order Table)
#GERDAU_BUCKET_WALLET_INPUT_PATH = "hana/lsa/adm/o2c/backlogopensales/backlogopsales_rpt_en_full"

In [ ]:
# Tvakt
GERDAU_BUCKET_TVAKT_INPUT_PATH = "global/masterdata/tvakt"

In [ ]:
# Vbap
GERDAU_BUCKET_VBAP_INPUT_PATH = "global/sales/vbap"

In [ ]:
SPG_QUERY_BUCKET_ATHENA = "SPG_QUERY/SPG_ATHENA"

In [ ]:
# Conditions
GERDAU_BUCKET_OPEN_ORDER_INPUT_PATH = "global/lsa/adm/o2c/backlogopensales_smartpricing/report_openorder_pricing"

In [ ]:
# SPG SAP Hana
SPG_INTEGRATION_INPUT_BUCKET_HANA = "SPG_FACTS/SPG_HANA/SPG_HANA.parquet"

In [ ]:
# SPG Regions
SPG_INTEGRATION_INPUT_BUCKET_REGION = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

In [ ]:
# SPG Products
SPG_INTEGRATION_INPUT_BUCKET_PRODUCTS = "SPG_DIMENSIONS/SPG_PRODUTOS/SPG_PD_PRODUTOS.parquet"

In [ ]:
# SPG Calendar
SPG_INPUT_BUCKET_CALENDAR = "SPG_GLOBAL/SUPPORT/SPG_SUPPORT_CALENDARIO/tb_SPG_SUPPORT_CALENDARIO.parquet"

In [ ]:
# pv
SPG_INPUT_BUCKET_SALES_PREVISION = "SPG_GLOBAL/SUPPORT/TB_SPG_SUPPORT_PV.parquet"

In [ ]:
# pex
SPG_INPUT_BUCKET_PEX = "SPG_GLOBAL/SUPPORT/TB_SPG_SUPPORT_PEX.parquet"

In [ ]:
# Alçada
SPG_INPUT_BUCKET_VERGE = "SPG_GLOBAL/SUPPORT/tb_SPG_SUPPORT_ALCADA.parquet"

In [ ]:
QUERY_WALLET = "SELECT * FROM "+ATHENA_HANA+".tb_o2c_backlogopsales_rpt_en_full_parquet inner join (SELECT DISTINCT max(dt_load) AS max_date FROM "+ATHENA_HANA+".tb_o2c_backlogopsales_rpt_en_full_parquet WHERE CustCountry LIKE '%Brazil%') ON dt_load = max_date WHERE CustCountry LIKE '%Brazil%'"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_VOLUME = "SPG_VOLUME/tb_SPG_VL_CURRENT.parquet"

In [ ]:
SPG_OUTPUT_BUCKET_SALES_PREV_HEAD = "SPG_VOLUME/SPG_PV/tb_SPG_VL_PV_HEAD.parquet"

In [ ]:
SPG_OUTPUT_BUCKET_FILTER = "SPG_VOLUME/tb_SPG_VL_FILTER.parquet"

## Boto3 Variables

In [ ]:
#S3 Configuration
S3_ATHENA_INPUT =  's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
S3_ATHENA_OUTPUT = 's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
region_name = os.environ['AWS_REGION']

In [ ]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY']

In [ ]:
aws_secret_access_key = os.environ['AWS_SECRET_KEY']

----------------

# Creating Defined Functions

In [ ]:
# Run Query

def run_query(query, database, s3_output):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

In [ ]:
def get_aws_path(query,database,s3_output):
    response = run_query(query, database, s3_output)
    file_query = response['QueryExecutionId']
    file_metadata = response['QueryExecutionId'] + '.metadata'
    return file_query

In [ ]:
# Wating for 300 seconds until the end of the upload

def wait_athena_load(Bucket, Key):
    time_to_wait = 300
    time_counter = 0

    while True:
        try:
            s3.meta.client.head_object(Bucket=Bucket,Key=Key)
        except ClientError:
            time.sleep(1)
            time_counter += 1
            if time_counter > time_to_wait:
                break
        else:
            break

In [3]:
# Replacing hifens by /
def remove_some_hifen(col_name):
    removed_chars = ("-")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Replacing dots by /
def remove_some_dots(col_name):
    removed_chars = (".")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Removing hifens
def replace_some_hifen(col_name):
    removed_chars = ("-")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Removing spaces
def replace_some_space(col_name):
    removed_chars = (" ")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Removing apostrophes
def replace_some_apostrophe(col_name):
    removed_chars = ("'")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------------

# Configuring Boto3

In [ ]:
#Athena Client Configuration

client = boto3.client('athena', 
    aws_access_key_id = aws_access_key_id, 
    aws_secret_access_key = aws_secret_access_key, 
    region_name = region_name )

In [ ]:
#S3 Resource Configuration

s3 = boto3.resource('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = region_name)

-----

# Importing Tables

In [8]:
# Vbap
df_vbap = spark.read.parquet("s3a://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_VBAP_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_vbap.write.partitionBy(144);
df_vbap = df_vbap.repartition(144);
df_vbap.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ABCCustomerClassification: string, BilDocNumber: string, BilDocItem: string, BILDOCDATE: string, BillingType: string, BillingTypeText: string, BusinessDivision: string, BusinessDivisionText: string, ShipToCity: string, SoldToCity: string, CompanyCode: string, CompanyCodeText: string, Conv33: string, SalesDoc_CorpGroup: string, CorporateGroupName: string, ShipToCountry: string, SoldToCountry: string, CustomerGroup: string, CustomerGroupName: string, CustomerGroup3: string, CustomerGroup3BRName: string, DateMonth: string, DateQuarter: string, DateWeek: string, DateYearMonth: string, DateYear: string, DistributionChannel: string, DistributionChannelName: string, MaterialDivision: string, MaterialDivisionName: string, Division: string, DivisionName: string, DocumentCurrency: string, EndMarket: string, GDD: string, GPD: string, GPDName: string, GPM: string, GPP: string, SoldToGrupoNISEBR: string, SalesDoc_InsideRep: string, SalesOrderInsideRepName: string, ShipToInsideRep: string,

In [10]:
# Tvakt
df_tvakt = spark.read.parquet("s3://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_TVAKT_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_tvakt.write.partitionBy(144);
df_tvakt = df_tvakt.repartition(144);
df_tvakt.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[IBGE_CITY_ID: bigint, IBGE_MICRO_ID: bigint, IBGE_UF_ID: bigint, IBGE_UF_NAME: string, IBGE_REGION_ID: bigint, IBGE_REGION_NAME: string, IBGE_REGION_ACRONYMS: string, IBGE_UF_ACRONYMS: string, IBGE_MESO_ID: bigint, IBGE_MESO_NAME: string, IBGE_MICRO_NAME: string, IBGE_CITY_NAME: string, SAP_COD: string, SAP_NAME: string, COUNTRY: string, CITY_ZF: int, RG_COD_IBGE: string, RG_REG_SAP: string, IBGE_SAP_CITY_ID: string, IBGE_SAP_CITY_NAME: string, BRANCH: string]

In [12]:
# Conditions
df_zeb1 = spark.read.parquet("s3a://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_OPEN_ORDER_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_zeb1.write.partitionBy(144);
df_zeb1 = df_zeb1.repartition(144);
df_zeb1.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[COD_MATERIAL: string, COD_ACTIVITY_SECTOR: string, MARKED_DELETION: string, DESC_MATERIAL: string, DESC_ACTIVITY_SECTOR: string, COD_GPD: string, SALES_ORG_COD: string, DESC_GPD: string, BAG: double, BAR: double, CS: double, DR: double, EA: double, FT: double, FT2: double, GLL: double, KAR: double, KG: double, KI: double, KWH: double, LB: double, LBF: double, M: double, M2: double, M3: double, MWH: double, PAA: double, PAK: double, QT: double, ROL: double, ST: double, TO: double, TON: double, YD2: double, YD3: double, ZBA: double, ZCJ: double, ZCT: double, ZFX: double, ZHT: double, ZSV: double, ZSW: double, ZUN: double, COD_HIERARCHY_1: string, DESC_HIERARCHY_1: string, COD_HIERARCHY_2: string, DESC_HIERARCHY_2: string, COD_HIERARCHY_3: string, DESC_HIERARCHY_3: string, COD_HIERARCHY_4: string, DESC_HIERARCHY_4: string, COD_HIERARCHY_5: string, DESC_HIERARCHY_5: string, COD_HIERARCHY_6: string, DESC_HIERARCHY_6: string, COD_HIERARCHY_7: string, DESC_HIERARCHY_7: string, COD_G

In [14]:
# SPG SAP Hana
df_hana = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_HANA)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_hana.write.partitionBy(144);
df_hana = df_hana.repartition(144);
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# SPG Regions
df_region = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_REGION)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_region.write.partitionBy(144);
df_region = df_region.repartition(144);
df_region.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# SPG Products
df_products = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_PRODUCTS)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_products.write.partitionBy(144);
df_products = df_products.repartition(144);
df_products.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{u'AcceptRanges': 'bytes', u'ContentType': 'binary/octet-stream', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'npLUgQZmAmfsd4dPVHESH5ZRimrurWrVIx3S0g8WkZW2pcEcfLVbxjhq9BVVUvlh2z7HIK5A2Qc=', 'RequestId': '43463A232B5C9907', 'HTTPHeaders': {'content-length': '792716903', 'x-amz-id-2': 'npLUgQZmAmfsd4dPVHESH5ZRimrurWrVIx3S0g8WkZW2pcEcfLVbxjhq9BVVUvlh2z7HIK5A2Qc=', 'accept-ranges': 'bytes', 'server': 'AmazonS3', 'last-modified': 'Fri, 06 Dec 2019 13:31:37 GMT', 'x-amz-request-id': '43463A232B5C9907', 'etag': '"83b8d0f7123dd467836ef4b7d8d48039-6"', 'date': 'Fri, 06 Dec 2019 13:32:28 GMT', 'content-type': 'binary/octet-stream'}}, u'LastModified': datetime.datetime(2019, 12, 6, 13, 31, 37, tzinfo=tzutc()), u'ContentLength': 792716903, u'ETag': '"83b8d0f7123dd467836ef4b7d8d48039-6"', u'Metadata': {}}

In [20]:
# SPG Calendar
df_calendar = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_CALENDAR)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_calendar.write.partitionBy(144);
df_calendar = df_calendar.repartition(144);
df_calendar.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# pv
df_pv = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_SALES_PREVISION)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[addvaldays: string, agreementid: string, blockresource: string, cgfinalcdesc: string, custcitynameship: string, confirmeddate: string, confirmedweek: string, contract: string, convertedunit: string, corporategroup: string, custcountry: string, createdby: string, creditissue: string, currency: string, custrollnumber: string, custcondgroup1: string, custcondgroup2: string, custcondgroup3: string, custcondgroup4: string, custcondgroup5: string, customergrp5: string, customerhier: string, custmatnumb: string, customer: string, customerpodate: string, cycleletter: string, dealcodejboss: string, dealname: string, deliverycalendar: string, deliverycalendardesc: string, deliverygrp: string, corporategroupname: string, custnameship: string, descdeliverybh: string, descdeliverybi: string, materialname: string, distchannel: string, division: string, divisiondesc: string, endmarket: string, finalapplication: string, fixedvaldate: string, goodsissuedate: string, grade: string, incoterms: 

In [23]:
df_pv.write.partitionBy(144);
df_pv = df_pv.repartition(144);
df_pv.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# pex
df_pex = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_PEX)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[NUMERIC_DATE: string, CURRENT_DATE: string, CURRENT_WEEK_NUMBER: string, CURRENT_MONTH: string, CURRENT_WEEK_NUMBER_MONTH: string, CURRENT_WEEKEND: string, LMONTH_DATE: string, LMONTH_WEEK_NUMBER: string, LAST_MONTH: string, LMONTH_WEEK_NUMBER_MONTH: string, LMONTH_WEEKEND: string, L2MONTHS_DATE: string, L2MONTHS_WEEK_NUMBER: string, LAST_2_MONTHS: string, L2MONTHS_WEEK_NUMBER_MONTH: string, L2MONTHS_WEEKEND: string, LYEAR_DATE: string, LYEAR_WEEK_NUMBER: string, LYEAR_MONTH: string, LYEAR_WEEK_NUMBER_MONTH: string, LYEAR_WEEKEND: string]

In [25]:
df_pex.write.partitionBy(144);
df_pex = df_pex.repartition(144);
df_pex.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Alçada
df_alcada = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_VERGE)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[MANDT: int, VBELN: string, POSNR: int, MATNR: string, MATWA: string, PMATN: string, CHARG: string, MATKL: string, ARKTX: string, PSTYV: string, POSAR: string, LFREL: string, FKREL: string, UEPOS: int, GRPOS: int, ABGRU: string, PRODH: string, ZWERT: double, ZMENG: double, ZIEME: string, UMZIZ: double, UMZIN: double, MEINS: string, SMENG: double, ABLFZ: double, ABDAT: string, ABSFZ: double, POSEX: string, KDMAT: string, KBVER: double, KEVER: double, VKGRU: string, VKAUS: string, GRKOR: int, FMENG: string, UEBTK: string, UEBTO: double, UNTTO: double, FAKSP: string, ATPKZ: string, RKFKF: string, SPART: string, GSBER: string, NETWR: double, WAERK: string, ANTLF: double, KZTLF: string, CHSPL: string, KWMENG: double, LSMENG: double, KBMENG: double, KLMENG: double, VRKME: string, UMVKZ: double, UMVKN: double, BRGEW: double, NTGEW: double, GEWEI: string, VOLUM: double, VOLEH: string, VBELV: string, POSNV: int, VGBEL: string, VGPOS: int, VOREF: string, UPFLU: string, ERLRE: string, LP

In [27]:
df_alcada.write.partitionBy(144);
df_alcada = df_alcada.repartition(144);
df_alcada.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Import CSV from View

athena_response = get_aws_path(QUERY_WALLET,ATHENA_SPG,S3_ATHENA_OUTPUT)

wait_athena_load(SPG_QUERY_BUCKET, SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[MANDT: int, SPRAS: string, AUART: string, BEZEI: string, TXT_BUTTON: string, DATA_LOAD_TIMESTAMP_PARQUET: string, DATA_LOAD_TIMESTAMP_RAW: string]

In [29]:
df_wallet = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Import CSV from View

df_wallet = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[GPD_COD: string, SALES_ORG_COD: string, MEASURE_UNIT: string, MONTH_YEAR: string, STATE: string, SALES_OFFICE: string, VOLUME: string, DESC_GPD: string, DESC_GPM: string]

In [31]:
# Wallet (Sales Order Table)
#df_wallet = spark.read.parquet("s3a://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_WALLET_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_wallet.write.partitionBy(144);
df_wallet = df_wallet.repartition(144);
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[GPD_COD: string, SALES_ORG_COD: string, VALUE_BRL: string, STATE: string, MONTH_YEAR: string, SALES_OFFICE: string, MEASURE_UNIT: string, VOLUME: string, DESC_GPD: string, DESC_GPM: string]

-------------

# Preparing ZEB1 Table

In [37]:
df_zeb1=df_zeb1.select(df_zeb1.VBAK_VBELN.alias("SALES_ORDER_NUMBER_drop")
                      ,df_zeb1.VBAP_POSNR.alias("SALES_ORDER_ITEM_drop")
                      ,df_zeb1.ZEB1.alias("ZEB1").cast("float"))\
               .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Preparing Regions Table

In [38]:
# Normalizing the column IBGE_SAP_CITY_NAME
df_region=df_region.withColumn("UPPER_NAME", upper(replace_some_apostrophe(replace_some_space(replace_some_hifen(df_region["IBGE_SAP_CITY_NAME"])))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# Selecting Necessary Columns
df_region=df_region.select(df_region.UPPER_NAME
                          ,df_region.BRANCH
                          ,df_region.IBGE_UF_ACRONYMS)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Selecting necessary columns from Ragions DateFrame
df_region2=df_region.select(df_region["IBGE_UF_ACRONYMS"].alias("STATE_drop")
                           ,df_region["BRANCH"]
                           ,upper(df_region["UPPER_NAME"]).alias("CITY_DROP"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_region3=df_region.select(df_region.IBGE_UF_ACRONYMS, df_region.BRANCH).filter(df_region.BRANCH.isNotNull()).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------

# Preparing Alcada Table

In [42]:
df_alcada=df_alcada.select(df_alcada.sales_org_cod.alias("SALES_ORG_COD")
                          ,df_alcada.material_cod.alias("MATERIAL_COD")
                          ,df_alcada.ibge_uf_acronyms.alias("ISSUING_STATE")
                          ,df_alcada.ALCADA.cast("float").alias("VERGE"))\
                    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----

# Preparing Products Table

In [43]:
# Selecting Necessary Columns
df_products=df_products.select(df_products.DESC_GPD, df_products.COD_GPD
                               ,df_products.DESC_GPM.alias("GPM_DESC")
                               ,df_products.COD_MATERIAL.alias("COD_MATERIAL_drop")
                               ,df_products.SALES_ORG_COD.alias("SALES_ORG_COD_drop"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------

# Preparing Calendar Table

In [44]:
# Normalizing Date columns
for col_name in ["CURRENT_DATE", "LMONTH_DATE", "L2MONTHS_DATE", "LYEAR_DATE"]:
    df_calendar = df_calendar.withColumn(col_name, remove_some_hifen(col_name))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# Transforming string columns to DateTime
for col_name in ["CURRENT_DATE", "LMONTH_DATE", "L2MONTHS_DATE", "LYEAR_DATE"]:
    df_calendar = df_calendar.withColumn(col_name, from_unixtime(unix_timestamp(col_name, 'dd/MM/yyyy')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Filtering Time Horizon (possible variation - current: 6 months)
df_calendar=df_calendar.filter(df_calendar['CURRENT_DATE']>=add_months(trunc(current_date(),"month"), -12)).filter(df_calendar['CURRENT_DATE']<=add_months(current_date(), 0)).distinct().dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------

# Preparing Vbap Table

In [47]:
df_vbap=df_vbap.select(df_vbap.VBELN.alias("SALES_ORDER_NUMBER_drop")
                     ,df_vbap.POSNR.alias("SALES_ORDER_ITEM_drop")
                     ,df_vbap.NETWR.alias("RLV").cast("float")
                     ,df_vbap.ZZICMS_IPI_T.alias("RBV").cast("float")
                     ,df_vbap.MWSBP.alias("TOTAL_TAXES").cast("float")
                     ,df_vbap.KWMENG.cast("float")
                     ,df_vbap.UMVKZ.cast("float")
                     ,df_vbap.UMVKN.cast("float"))\
              .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_vbap=df_vbap.withColumn("REQUESTED_QUANTITY", ((col("KWMENG")*col("UMVKZ"))/col("UMVKN")).cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df_vbap=df_vbap.withColumn("QUANTITY_TON", col("REQUESTED_QUANTITY").cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_vbap=df_vbap.drop("KWMENG", "UMVKZ", "UMVKN")\
              .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Preparing Wallet Table

In [51]:
# Filtering GAB Sales Organizations
df_wallet=df_wallet.filter(df_wallet.salesorg.like('%BRIN%') |
                           df_wallet.salesorg.like('%BRIO%') |
                           df_wallet.salesorg.like('%BRDI%') |
                           df_wallet.salesorg.like('%BRDO%') |
                           df_wallet.salesorg.like('%BRCC%') |
                           df_wallet.salesorg.like('%BRCO%') |
                           df_wallet.salesorg.like('%BRCG%') |
                           df_wallet.salesorg.like('%BRGO%'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Filtering necessary types of orders
df_wallet=df_wallet.filter(df_wallet.ordertype.like('%YORB%') |
                             df_wallet.ordertype.like('%ZORB%') |
                             df_wallet.ordertype.like('%ZVOI%') |
                             df_wallet.ordertype.like('%YVOI%') |
                             df_wallet.ordertype.like('%ZCNC%') |
                             df_wallet.ordertype.like('%ZVOD%') |
                             df_wallet.ordertype.like('%YCNC%') |
                             df_wallet.ordertype.like('%ZNFC%') |
                             df_wallet.ordertype.like('%ZNNC%') |
                             df_wallet.ordertype.like('%ZNNL%') |
                             df_wallet.ordertype.like('%ZRNP%') |
                             df_wallet.ordertype.like('%ZRLC%') |
                             df_wallet.ordertype.like('%ZDNC%') |
                             df_wallet.ordertype.like('%ZDNM%') |
                             df_wallet.ordertype.like('%ZNFD%') |
                             df_wallet.ordertype.like('%B4%') |
                             df_wallet.ordertype.like('%B3%') |
                             df_wallet.ordertype.like('%ZCUP%') |
                             df_wallet.ordertype.like('%#%') |
                             df_wallet.ordertype.like('%YVOD%'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# Normalizing Date columns
for col_name in ["dt_load", "ordercreationdate", "confirmeddate"]:
    df_wallet = df_wallet.withColumn(col_name, from_unixtime(unix_timestamp(remove_some_dots(col_name), 'yyyy/MM/dd')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# Renaming and selecting the columns
df_wallet = df_wallet.select(col("salesorg").alias("SALES_ORG_COD"),
                             col("custregionship").alias("ISSUING_STATE"),
                             col("customer").alias("CUSTOMER_COD"),
                             col("custname").alias("CUSTOMER_DESC"),
                             col("corporategroupname").alias("CUSTOMER_GROUP"),
                             col("item").alias("BILLING_ITEM"),
                             col("custcitynameship").alias("ISSUING_CITY"),
                             col("material").alias("MATERIAL_COD"),
                             col("materialname").alias("MATERIAL_DESC"),
                             col("salesoffice").alias("SALES_OFFICE_COD"),
#                             col("confirmedqtt").alias("QUANTITY_TON"),
                             col("materialgroup1").alias("GPD"),                    
                             col("deliveredquantity").alias("DELIVERED_QUANTITY").cast("float"),
                             col("reqqttconv").alias("REQ_QTT_CONV"),
                             col("zd06"),
                             col("zd13"),
                             col("zd14"),
                             col("manualextra").alias("ZSU2"),
                             col("brFinanceextra").alias("ZEAF"),
                             col("confirmeddate").alias("CONFIRMED_ORDER_DATE"),
                             col("ordercreationdate").alias("CREATED_ORDER_DATE"),
                             col("ordercreationdate").alias("SALES_ORDER_DATE"),
                             col("goodsissuedate").alias("ISSUING_DATE"),
                             col("statusoverall").alias("ORDER_STATUS"),
                             col("order").alias("BILLING_NUMBER"),
                             col("salesgroupdesc").alias("SALES_GROUP_DESC"),
                             col("ordertype").alias("ORDER_TYPE"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# Creating "ZEB1" Column
df_wallet = df_wallet.join(df_zeb1, (df_wallet["BILLING_NUMBER"].cast("int")==df_zeb1["SALES_ORDER_NUMBER_drop"].cast("int"))&
                                    (df_wallet["ORDER_TYPE"].cast("int")==df_zeb1["SALES_ORDER_ITEM_drop"].cast("int")), how='left')\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, GPD: string, DELIVERED_QUANTITY: float, REQ_QTT_CONV: string, zd06: string, zd13: string, zd14: string, ZSU2: string, ZEAF: string, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, SALES_ORDER_NUMBER_drop: string, SALES_ORDER_ITEM_drop: string, ZEB1: float]

In [57]:
df_wallet = df_wallet.drop("SALES_ORDER_NUMBER_drop")\
                     .drop("SALES_ORDER_ITEM_drop")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
# Renaming and selecting the columns
# Filtering Language
df_tvakt = df_tvakt.select(col("spras").alias("LANGUAGE"),
                             col("bezei").alias("ORDER_TYPE_DESC"),
                             col("auart").alias("ORDER_TYPE_COD"))
df_tvakt = df_tvakt.filter(df_tvakt.LANGUAGE==('P'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# Creating "RLV" and TOTAL_TAXES Columns
df_wallet = df_wallet.join(df_vbap, (df_wallet["BILLING_NUMBER"].cast("int")==df_vbap["SALES_ORDER_NUMBER_drop"].cast("int"))&
                                   (df_wallet["BILLING_ITEM"].cast("int")==df_vbap["SALES_ORDER_ITEM_drop"].cast("int")), how='inner')\
                    .drop("SALES_ORDER_NUMBER_drop")\
                    .drop("SALES_ORDER_ITEM_drop")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, GPD: string, DELIVERED_QUANTITY: float, REQ_QTT_CONV: string, zd06: string, zd13: string, zd14: string, ZSU2: string, ZEAF: string, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, ZEB1: float, RLV: float, RBV: float, TOTAL_TAXES: float, REQUESTED_QUANTITY: float, QUANTITY_TON: float]

In [61]:
df_wallet=df_wallet.withColumn("PRACTICED_PRICE", col("RBV"))\
                   .withColumn("PRACTICED_PRICE_I", col("RBV"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
# Correcting "Ouro Branco" Sales Org 
    # BRIO -> BRIN
    # BRCO -> BRCC
    # BRDO -> BRDI
    # BRGO -> BRCG
df_wallet=df_wallet.withColumn("SALES_ORG_COD", when(df_wallet.SALES_ORG_COD.like("BRCO"), "BRCC")\
                                                    .otherwise(when(df_wallet.SALES_ORG_COD.like("BRGO"), "BRCG")\
                                                              .otherwise(when(df_wallet.SALES_ORG_COD.like("BRIO"), "BRIN")\
                                                                        .otherwise(when(df_wallet.SALES_ORG_COD.like("BRDO"), "BRDI")
                                                                                  .otherwise(df_wallet.SALES_ORG_COD)))))\
                   .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# Creating "OV_Ajust" support column 
df_wallet=df_wallet.withColumn("OV_AJUST", df_wallet.SALES_ORG_COD)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
# Correcting Practiced Price Field
df_wallet=df_wallet.withColumn("CUSTOMER_GROUP", when(df_wallet.CUSTOMER_GROUP.isNull()|
                                                  df_wallet.CUSTOMER_GROUP.like("GRUPO DE RENTABILIDADE GENÉRICO")
                                                  ,col("CUSTOMER_DESC")).otherwise(col("CUSTOMER_GROUP")))\
                    .withColumn("BILLING_DATE",lit(""))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
# Converting numeric(and text) columns to float 
for col_name in ["ZD06",
                 "ZD13",
                 "ZD14",
                 "ZSU2",
                 "ZEAF"]:
    df_wallet = df_wallet.withColumn(col_name, ((col(col_name).cast('float')*(abs(((col("REQUESTED_QUANTITY"))/col("REQ_QTT_CONV")))))/100).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Converting "QUANTITY_TON" from Kg to Tons
df_wallet = df_wallet.withColumn("QUANTITY_TON", (col("QUANTITY_TON")/1000).cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# Creating "PO" column
df_wallet=df_wallet.withColumn("PO", col("RBV")
                                     -abs(col("ZSU2")) 
                                     +abs(col("ZD13"))     
                                     +abs(col("ZD06"))
                                     +abs(col("ZD14")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
# Merge wallet and alcada tables
df_wallet=df_wallet.join(df_alcada.select(df_alcada.SALES_ORG_COD.alias("SALES_ORG_COD_drop")
                                         ,df_alcada.MATERIAL_COD.alias("MATERIAL_COD_drop")
                                         ,df_alcada.ISSUING_STATE.alias("ISSUING_STATE_drop")
                                         ,df_alcada.VERGE)
                         ,(trim(df_wallet.SALES_ORG_COD) == trim(col("SALES_ORG_COD_drop"))) &
                          (df_wallet.MATERIAL_COD.cast("int") == col("MATERIAL_COD_drop").cast("int")) &
                          (trim(df_wallet.ISSUING_STATE) == trim(col("ISSUING_STATE_drop"))), how="left")\
                    .drop("SALES_ORG_COD_drop"
                         ,"MATERIAL_COD_drop"
                         ,"ISSUING_STATE_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df_wallet = df_wallet.fillna({'VERGE':4})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# Creating "VERGE" column
df_wallet=df_wallet.withColumn("VERGE", col("VERGE").cast('float'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
df_wallet.write.partitionBy("SALES_ORG_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df_wallet = df_wallet.repartition("SALES_ORG_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, GPD: string, DELIVERED_QUANTITY: float, REQ_QTT_CONV: string, ZD06: float, ZD13: float, ZD14: float, ZSU2: float, ZEAF: float, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, ZEB1: float, RLV: float, RBV: float, TOTAL_TAXES: float, REQUESTED_QUANTITY: float, QUANTITY_TON: float, PRACTICED_PRICE: float, PRACTICED_PRICE_I: float, OV_AJUST: string, BILLING_DATE: string, PO: float, ALCADA: float]

In [74]:
# Creating "TOTAL_WALLET" column
df_wallet=df_wallet.withColumn("TOTAL_WALLET", ((abs(col("REQUESTED_QUANTITY")) - abs((col("DELIVERED_QUANTITY")*col("REQUESTED_QUANTITY"))/col("REQ_QTT_CONV")))/1000).cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df_wallet=df_wallet.drop("REQ_QTT_CONV")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
# Creating "POLICY_DEVIATION" column
df_wallet = df_wallet.withColumn("POLICY_DEVIATION", (when(abs((col("ZD06")+col("ZD13")+col("ZD14")))>4,abs(col("ZD06")+col("ZD13")+col("ZD14"))-col("VERGE")).otherwise(0)).cast("float"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
# Creating "POLICY_DEVIATION_REAIS" column
df_wallet = df_wallet.withColumn("POLICY_DEVIATION_REAIS", col("RBV")*col("POLICY_DEVIATION"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
# Creating Support Columns
df_wallet = df_wallet.withColumn("PERC_VOL", lit(0).cast('double'))\
                     .withColumn("PV_BUDGET", lit(0).cast('double'))\
                     .withColumn("PV_DONE", lit(0).cast('double'))\
                     .withColumn("MANUFACTURER", lit("Gerdau - Carteira"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# Getting GPM_DESC Field
df_wallet=df_wallet.join(df_products, (trim(df_products.SALES_ORG_COD_drop)==trim(df_wallet.SALES_ORG_COD))&
                                  (df_products.COD_MATERIAL_drop.cast("int")==df_wallet.MATERIAL_COD.cast("int")), how="left")\
                   .withColumnRenamed("DESC_GPD", "GPD_DESC")\
                   .withColumnRenamed("COD_GPD", "GPD_COD")\
                   .drop("COD_MATERIAL_drop")\
                   .drop("SALES_ORG_COD_drop")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df_wallet=df_wallet.withColumn("GPD_COD", when(df_wallet.GPD_COD.isNull(), col("GPD"))\
                               .otherwise(df_wallet.GPD_COD))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
df_wallet=df_wallet.drop("GPD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
df_wallet.write.partitionBy(144);
df_wallet = df_wallet.repartition(144);

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, DELIVERED_QUANTITY: float, ZD06: float, ZD13: float, ZD14: float, ZSU2: float, ZEAF: float, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, ZEB1: float, RLV: float, RBV: float, TOTAL_TAXES: float, REQUESTED_QUANTITY: float, QUANTITY_TON: float, PRACTICED_PRICE: float, PRACTICED_PRICE_I: float, OV_AJUST: string, BILLING_DATE: string, PO: float, ALCADA: float, TOTAL_WALLET: float, DESVIO_POL: float, DESVIO_POL_REAIS: float, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, GPD_DESC: string, GPD_COD: string, GPM_DESC: string]

In [84]:
# Creating "BRANCH" Column
df_wallet = df_wallet.join(df_region2, (replace_some_hifen(replace_some_space(replace_some_apostrophe(df_wallet["ISSUING_CITY"]))) == replace_some_hifen(replace_some_space(replace_some_apostrophe(df_region2["CITY_DROP"])))) &
                                      (trim(df_wallet["ISSUING_STATE"]) == trim(df_region2["STATE_drop"])), how='left')\
                     .drop("STATE_drop")\
                     .drop("CITY_DROP")\
                     .withColumn("KEY_brc", concat('SALES_ORG_COD'
                                                   ,'GPD_COD'
                                                   ,'ISSUING_STATE'
                                                   ,'BRANCH'))\
                     .withColumn("KEY_st", concat('SALES_ORG_COD'
                                                  ,'GPD_COD'
                                                  ,'ISSUING_STATE'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
df_wallet.write.partitionBy(144);
df_wallet = df_wallet.repartition(144);

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, DELIVERED_QUANTITY: float, ZD06: float, ZD13: float, ZD14: float, ZSU2: float, ZEAF: float, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, ZEB1: float, RLV: float, RBV: float, TOTAL_TAXES: float, REQUESTED_QUANTITY: float, QUANTITY_TON: float, PRACTICED_PRICE: float, PRACTICED_PRICE_I: float, OV_AJUST: string, BILLING_DATE: string, PO: float, ALCADA: float, TOTAL_WALLET: float, DESVIO_POL: float, DESVIO_POL_REAIS: float, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, GPD_DESC: string, GPD_COD: string, GPM_DESC: string, BRANCH: string, KEY_brc: string, KEY_st: string]

In [87]:
df_wallet = df_wallet.withColumn("BRANCH", when(df_wallet.BRANCH.isNull(), lit("Filial Não Encontrada")).otherwise(df_wallet.BRANCH))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
df_wallet_groupby1=df_wallet.groupby("SALES_ORG_COD", "GPD_COD", "ISSUING_STATE", "BRANCH").sum("QUANTITY_TON")\
                            .withColumnRenamed("sum(QUANTITY_TON)", 'QUANTITY_TON_BRC')\
                            .withColumn("KEY_drop", concat('SALES_ORG_COD'
                                                           ,'GPD_COD'
                                                           ,'ISSUING_STATE'
                                                           ,'BRANCH'))\
                            .withColumnRenamed("SALES_ORG_COD", 'SALES_ORG_COD_drop')\
                            .withColumnRenamed("GPD_COD", 'GPD_COD_drop')\
                            .withColumnRenamed("ISSUING_STATE", 'ISSUING_STATE_drop')\
                            .withColumnRenamed("BRANCH", 'BRANCH_drop')\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
df_wallet = df_wallet.join(df_wallet_groupby1,df_wallet.KEY_brc==df_wallet_groupby1.KEY_drop, how='inner')\
                     .drop("STATE_drop")\
                     .drop("CITY_DROP")\
                     .drop("KEY_drop")\
                     .drop("ISSUING_STATE_drop")\
                     .drop("GPD_COD_drop")\
                     .drop("BRANCH_drop")\
                     .drop("SALES_ORG_COD_drop")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df_wallet.write.partitionBy(144);
df_wallet = df_wallet.repartition(144);

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, DELIVERED_QUANTITY: float, ZD06: float, ZD13: float, ZD14: float, ZSU2: float, ZEAF: float, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, ZEB1: float, RLV: float, RBV: float, TOTAL_TAXES: float, REQUESTED_QUANTITY: float, QUANTITY_TON: float, PRACTICED_PRICE: float, PRACTICED_PRICE_I: float, OV_AJUST: string, BILLING_DATE: string, PO: float, ALCADA: float, TOTAL_WALLET: float, DESVIO_POL: float, DESVIO_POL_REAIS: float, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, GPD_DESC: string, GPD_COD: string, GPM_DESC: string, BRANCH: string, KEY_brc: string, KEY_st: string, QUANTITY

In [92]:
df_wallet_groupby2=df_wallet.groupby("SALES_ORG_COD", "GPD_COD", "ISSUING_STATE").sum("QUANTITY_TON")\
                            .withColumnRenamed("sum(QUANTITY_TON)", 'QUANTITY_TON_ST')\
                            .withColumn("KEY_drop", concat('SALES_ORG_COD'
                                                           ,'GPD_COD'
                                                           ,'ISSUING_STATE'))\
                            .withColumnRenamed("SALES_ORG_COD", 'SALES_ORG_COD_drop')\
                            .withColumnRenamed("GPD_COD", 'GPD_COD_drop')\
                            .withColumnRenamed("ISSUING_STATE", 'ISSUING_STATE_drop')\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
df_wallet = df_wallet.join(df_wallet_groupby2,df_wallet.KEY_st==df_wallet_groupby2.KEY_drop, how='inner')\
                     .drop("STATE_drop")\
                     .drop("CITY_DROP")\
                     .drop("KEY_drop")\
                     .drop("KEY_brc")\
                     .drop("KEY_st")\
                     .drop("SALES_ORG_COD_drop")\
                     .drop("GPD_COD_drop")\
                     .drop("ISSUING_STATE_drop")\
                     .drop("BRANCH_drop")\
                     .drop("SALES_ORG_COD_drop")\
                     .drop("REGION")\
                     .drop("ZSU2")\
                     .drop("ZEAF")\
                     .drop("ZEB1")\
                     .drop("REQUESTED_QUANTITY")\
                     .drop("DELIVERED_QUANTITY")\
                     .drop("TOTAL_TAXES")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
df_wallet.write.partitionBy(144);
df_wallet = df_wallet.repartition(144);

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
df_wallet.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_ITEM: string, ISSUING_CITY: string, MATERIAL_COD: string, MATERIAL_DESC: string, SALES_OFFICE_COD: string, ZD06: float, ZD13: float, ZD14: float, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, SALES_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, BILLING_NUMBER: string, SALES_GROUP_DESC: string, ORDER_TYPE: string, RLV: float, RBV: float, QUANTITY_TON: float, PRACTICED_PRICE: float, PRACTICED_PRICE_I: float, OV_AJUST: string, BILLING_DATE: string, PO: float, ALCADA: float, TOTAL_WALLET: float, DESVIO_POL: float, DESVIO_POL_REAIS: float, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, GPD_DESC: string, GPD_COD: string, GPM_DESC: string, BRANCH: string, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double]

-------------

# Preparing SPG HANA

In [96]:
# Filtering Sales Org and Billing Type
df_hana=df_hana.filter(df_hana.SalesOrg.like('%BRIN%') |
                       df_hana.SalesOrg.like('%BRIO%') |
                       df_hana.SalesOrg.like('%BRDI%') |
                       df_hana.SalesOrg.like('%BRDO%') |
                       df_hana.SalesOrg.like('%BRCC%') |
                       df_hana.SalesOrg.like('%BRCO%') |
                       df_hana.SalesOrg.like('%BRCG%') |
                       df_hana.SalesOrg.like('%BRGO%'))\
               .filter(df_hana.BillingType.like('%ZF2B%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
# Correcting "Ouro Fino" Sales Org 
    # BRIO -> BRIN
    # BRCO -> BRCC
    # BRDO -> BRDI
    # BRGO -> BRCG
df_hana = df_hana.withColumn("SalesOrg", when(df_hana.SalesOrg.like("BRCO"), "BRCC")\
                                              .otherwise(when(df_hana.SalesOrg.like("BRGO"), "BRCG")\
                                                        .otherwise(when(df_hana.SalesOrg.like("BRIO"), "BRIN")\
                                                                  .otherwise(when(df_hana.SalesOrg.like("BRDO"), "BRDI")
                                                                            .otherwise(df_hana.SalesOrg)))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
# Creating OV_AJUST Field
df_hana=df_hana.withColumn("OV_AJUST", df_hana.SalesOrg)\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
# Getting GPM_DESC Field
df_hana=df_hana.join(df_products, (trim(df_products.SALES_ORG_COD_drop)==trim(df_hana.SalesOrg))&
                                  (df_products.COD_MATERIAL_drop.cast("int")==df_hana.Material.cast("int")), how="left")\
               .drop("DESC_GPD")\
               .drop("COD_GPD")\
               .drop("COD_MATERIAL_drop")\
               .drop("SALES_ORG_COD_drop")\
               .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ABCCustomerClassification: string, BilDocNumber: string, BilDocItem: string, BILDOCDATE: string, BillingType: string, BillingTypeText: string, BusinessDivision: string, BusinessDivisionText: string, ShipToCity: string, SoldToCity: string, CompanyCode: string, CompanyCodeText: string, Conv33: string, SalesDoc_CorpGroup: string, CorporateGroupName: string, ShipToCountry: string, SoldToCountry: string, CustomerGroup: string, CustomerGroupName: string, CustomerGroup3: string, CustomerGroup3BRName: string, DateMonth: string, DateQuarter: string, DateWeek: string, DateYearMonth: string, DateYear: string, DistributionChannel: string, DistributionChannelName: string, MaterialDivision: string, MaterialDivisionName: string, Division: string, DivisionName: string, DocumentCurrency: string, EndMarket: string, GDD: string, GPD: string, GPDName: string, GPM: string, GPP: string, SoldToGrupoNISEBR: string, SalesDoc_InsideRep: string, SalesOrderInsideRepName: string, ShipToInsideRep: string,

In [101]:
# Correcting GPM_DESC Column
df_hana=df_hana.withColumn("GPM", when(df_hana.GPM_DESC.isNull(), df_hana.GPM)\
                                           .otherwise(df_hana.GPM_DESC))\
               .drop("GPM_DESC")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
#choosing specifics GPD's (business call)
df_hana = df_hana.filter(df_hana.GPDName.like('%BR-PERFIS ESTRUTURAIS%') |
                        df_hana.GPDName.like('%BR-CA50%') |
                        df_hana.GPDName.like('%BR-CA60%') |
                        df_hana.GPDName.like('%BR-RECOZIDO%') |
                        df_hana.GPDName.like('%BR-TEL TUBO%') |
                        df_hana.GPDName.like('%BR-TELA P/ CONCRETO%') |
                        df_hana.GPDName.like('%BR-CORTE E DOBR%') |
                        df_hana.GPDName.like('%BR-CONSTRUCAO CIVIL%') |
                        df_hana.GPDName.like('%BR-CA25%') |
                        df_hana.GPDName.like('%BR-PERFIS COMERCIAIS%') |
                        df_hana.GPDName.like('%BR-BTG%') |
                        df_hana.GPDName.like('%BR-MALHA POP%') |
                        df_hana.GPDName.like('%BR-TELA COLUNA%') |
                        df_hana.GPDName.like('%BR-TRELIÇA%') |
                        df_hana.GPDName.like('%BR-B.TREFILADA%') |
                        df_hana.GPDName.like('%BR-OVALADO%') |
                        df_hana.GPDName.like('%BR-GALVANIZADO%') |
                        df_hana.GPDName.like('%BR-BARRAS COMERCIAIS%') |
                        df_hana.GPDName.like('%BR-CAIXOTARIA%') |
                        df_hana.GPDName.like('%BR-ARTEFATOS%') |
                        df_hana.GPDName.like('%BR-FARPADO%') |
                        df_hana.GPDName.like('%BR-ELETRODO%') |
                        df_hana.GPDName.like('%BR-SOLDA - MIG%') |
                        df_hana.GPDName.like('%BR-CANTON A572%') |
                        df_hana.GPDName.like('%BR-GGS%') |
                        df_hana.GPDName.like('%BR-ARMADO%') |
                        df_hana.GPDName.like('%BR-CORDOALHA%') |
                        df_hana.GPDName.like('%BR-ESTACA PRANCHA%') |
                        df_hana.GPDName.like('%BR-ARAME PREGO%') |
                        df_hana.GPDName.like('%BR-CABEÇA DUPLA%') |
                        df_hana.GPDName.like('%BR-CORDOALHA AGRO%') |
                        df_hana.GPDName.like('%BR-GRAMPO%') |
                        df_hana.GPDName.like('%BR-COBREADOS%') |
                        df_hana.GPDName.like('%BR-CHAPA LQ%') |
                        df_hana.GPDName.like('%BR-UDC%') |
                        df_hana.GPDName.like('%BR-CHAPA ZN%') |
                        df_hana.GPDName.like('%BR-TELHA AZ%') |
                        df_hana.GPDName.like('%BR-TUBO ZN%') |
                        df_hana.GPDName.like('%BR-MARCENARIA%') |
                        df_hana.GPDName.like('%BR-PREGOES%') |
                        df_hana.GPDName.like('%BR-TELHEIRO%') |
                        df_hana.GPDName.like('%BR-COLUNA%') |
                        df_hana.GPDName.like('%BR-ESTRIBO%') |
                        df_hana.GPDName.like('%BR-ACESSORIOS%') |
                        df_hana.GPDName.like('%BR-CHAPA LCG%') |
                        df_hana.GPDName.like('%BR-CHAPA LF%') |
                        df_hana.GPDName.like('%BR-TUBO LF%') |
                        df_hana.GPDName.like('%BR-CHAPA LQ PISO%') |
                        df_hana.GPDName.like('%BR-BOBININHA%') |
                        df_hana.GPDName.like('%BR-ESPECIAIS%') |
                        df_hana.GPDName.like('%BR-BOBINA LQ%') |
                        df_hana.GPDName.like('%BR-FITA LQ%') |
                        df_hana.GPDName.like('%BR-BOBINA AZ%') |
                        df_hana.GPDName.like('%BR-AÇOS ESPECIAIS%') |
                        df_hana.GPDName.like('%BR-PARAFUSOS%') |
                        df_hana.GPDName.like('%BR-CIMENTO%') |
                        df_hana.GPDName.like('%BR-TUBO LQ%') |
                        df_hana.GPDName.like('%BR-TELHA ZN%') |
                        df_hana.GPDName.like('%BR-BTC GLV CP%') |
                        df_hana.GPDName.like('%BR-BOBINA ZN%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA MÉDIA%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA PESADA%') |
                        df_hana.GPDName.like('%BR-FITA LF%') |
                        df_hana.GPDName.like('%BR-FITA AZ%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA LEVE%') |
                        df_hana.GPDName.like('%BR-ALAMBRADO%') |
                        df_hana.GPDName.like('%BR-SAPATA%') |
                        df_hana.GPDName.like('%BR-MOURÃO%') |
                        df_hana.GPDName.like('%BR-ATC CLARO IND%') |
                        df_hana.GPDName.like('%BR-POLIDO%') |
                        df_hana.GPDName.like('%BR-PERFIL BENEFICIADO%') |
                        df_hana.GPDName.like('%BR-BOBINA LF%') |
                        df_hana.GPDName.like('%BR-S-BARRA CTT%') |
                        df_hana.GPDName.like('%BR-CHAPA AZ%') |
                        df_hana.GPDName.like('%BR-FITA ZN%') |
                        df_hana.GPDName.like('%BR-LDA%') |
                        df_hana.GPDName.like('%BR-PIATINA CLARA%') |
                        df_hana.GPDName.like('%BR-S-BARRA DESCASCADA%') |
                        df_hana.GPDName.like('%BR-MESH%') |
                        df_hana.GPDName.like('%BR-S-BARRA FORJADA FINA%') |
                        df_hana.GPDName.like('%BR-S-BARRA RETIFICADA%') |
                        df_hana.GPDName.like('%BR-S-BARRA TREFILADA%') |
                        df_hana.GPDName.like('%BR-PLACA%') |
                        df_hana.GPDName.like('%BR-COIL%') |
                        df_hana.GPDName.like('%BR-S-FERRAMENTA%') |
                        df_hana.GPDName.like('%BR-S-FIO MÁQUINA STT%') |
                        df_hana.GPDName.like('%BR-ATC CLARO MOL COL%') |
                        df_hana.GPDName.like('%BR-ATC GLV CP%') |
                        df_hana.GPDName.like('%BR-B.CHATA LONGARINA%') |
                        df_hana.GPDName.like('%BR-BT FORJARIA BLOCOS%') |
                        df_hana.GPDName.like('%BR-TARUGO%') |
                        df_hana.GPDName.like('%BR-DRAWING%') |
                        df_hana.GPDName.like('%BR-ATC CLARO ENF ALG%') |
                        df_hana.GPDName.like('%BR-PIATINA GLV%') |
                        df_hana.GPDName.like('%BR-PERFIL DORMENTE%') |
                        df_hana.GPDName.like('%BR-ATC%') |
                        df_hana.GPDName.like('%BR-CHQ%') |
                        df_hana.GPDName.like('%BR-ALMA DE ELETRODO%') |
                        df_hana.GPDName.like('%BR-FIO MAQUINA%') |
                        df_hana.GPDName.like('%BR-CHQ BORO%') |
                        df_hana.GPDName.like('%BR-PERFIL GUIA ELEV%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
#choosiin
df_hana = df_hana.filter(df_hana.GPM.like('%PERFIS_ESTRUTURAIS%') |
                        df_hana.GPM.like('%VERGALHAO%') |
                        df_hana.GPM.like('%AMPLIADOS%') |
                        df_hana.GPM.like('%CORTE_DOBRA%') |
                        df_hana.GPM.like('%PREGOS%') |
                        df_hana.GPM.like('%B&P%') |
                        df_hana.GPM.like('%ARAMES_AGRO%') |
                        df_hana.GPM.like('%ARAMES_IND%') |
                        df_hana.GPM.like('%PLANOS_LQ%') |
                        df_hana.GPM.like('%PLANOS_REVESTIDOS%') |
                        df_hana.GPM.like('%PLANOS_LF%') |
                        df_hana.GPM.like('%PLANOS_LCG%') |
                        df_hana.GPM.like('%FIO_MAQUINA%') |
                        df_hana.GPM.like('%BT_FORJARIA%') |
                        df_hana.GPM.like('%PLACAS%') |
                        df_hana.GPM.like('%TARUGO%') | 
                        df_hana.GPM.isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
# Selecting Necessary Columns
df_hana=df_hana.select(df_hana.SoldToCustomer.alias("CUSTOMER_COD")
                       ,df_hana.SoldToCustomerName.alias("CUSTOMER_DESC")
                       ,df_hana.CorporateGroupName.alias("CUSTOMER_GROUP")
                       ,df_hana.BilDocNumber.alias("BILLING_NUMBER")
                       ,df_hana.BilDocItem.alias("BILLING_ITEM")
                       ,df_hana.BILDOCDATE.alias("BILLING_DATE")
                       ,df_hana.SalesOrg.alias("SALES_ORG_COD")
                       ,df_hana.OV_AJUST
                       ,df_hana.SalesOrderDate.alias("SALES_ORDER_DATE")
                       ,df_hana.SalesGroupName.alias("SALES_GROUP_DESC")
                       ,df_hana.SoldToCity.alias("ISSUING_CITY")
                       ,df_hana.SoldToState.alias("ISSUING_STATE")
                       ,df_hana.GPD.alias("GPD_COD")
                       ,df_hana.GPM.alias("GPM_DESC")
                       ,df_hana.GPDName.alias("GPD_DESC")
                       ,df_hana.Material.alias("MATERIAL_COD").cast("int").cast("string")
                       ,df_hana.SalesOffice.alias("SALES_OFFICE_COD")
                       ,df_hana.MaterialName.alias("MATERIAL_DESC")
                       ,df_hana.SalesDocType.alias("ORDER_TYPE")
                       ,df_hana.MA_BillingQuantity_P.alias("QUANTITY_TON")
                       ,df_hana.DP_CM_PRECOPRATICADO_D.alias("PRACTICED_PRICE")
                       ,df_hana.DA_CM_CD_ZD05_D.alias("ZD05")
                       ,df_hana.AA_CM_CD_ZEAF_D.alias("ZEAF")
                       ,df_hana.MA_CM_CD_BX41_D.alias("BX41")
                       ,df_hana.MA_CM_RBV_D.alias("RBV")                
                       ,df_hana.MA_CM_RLV_D.alias("RLV")
                       ,df_hana.MA_CM_PRECOBASE_ALCADA_P.alias("VERGE")                       
                       ,df_hana.DA_CM_CD_ZD01_D.alias("ZD01")
                       ,df_hana.DA_CM_CD_ZD02_D.alias("ZD02")
                       ,df_hana.DA_CM_CD_ZD03_D.alias("ZD03")
                       ,df_hana.DA_CM_CD_ZD04_D.alias("ZD04")                       
                       ,df_hana.DA_CM_CD_ZD06_D.alias("ZD06")
                       ,df_hana.DM_CM_CD_ZD13_D.alias("ZD13")
                       ,df_hana.MA_CM_PRECOLISTA_D.alias("LIST_PRICE")
                       ,df_hana.AA_CM_CD_ZEB1_D.alias("ZEB1")
                       ,df_hana.AA_CM_CD_ZEB2_D.alias("ZEB2")
                       ,df_hana.AM_CM_CD_ZSU2_D.alias("ZSU2")    
                       ,df_hana.MA_CM_DESVIOPOLITICAALCADA_D.alias("POLICY_DEVIATION")
                       ,df_hana.MA_CM_DESVIOPOLITICAALCADA_D.alias("POLICY_DEVIATION_REAIS")
                       ,df_hana.DM_CM_CD_ZD14_D.alias("ZD14"))\
                .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
df_hana=df_hana.withColumn("SALES_ORDER_DATE", concat(substring("SALES_ORDER_DATE", 1, 4)
                                                      ,lit("/")
                                                      ,substring("SALES_ORDER_DATE", -4, 2)
                                                      ,lit("/")
                                                      ,substring("SALES_ORDER_DATE", 7, 8)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
df_hana = df_hana.withColumn("SALES_ORDER_DATE", from_unixtime(unix_timestamp(col("SALES_ORDER_DATE"), 'yyyy/MM/dd')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
# Converting numeric values from "Text" (String) to Float 
for col_name in ["QUANTITY_TON"
                 ,"PRACTICED_PRICE"
                 ,"ZD05"
                 ,"ZEAF"
                 ,"BX41"
                 ,"RBV"
                 ,"RLV"
                 ,"VERGE"                 
                 ,"ZD01"
                 ,"ZD02"
                 ,"ZD03"
                 ,"ZD04"
                 ,"ZD06"
                 ,"ZD13"
                 ,"PRICE_LIST"
                 ,"ZEB1"
                 ,"ZEB2"
                 ,"ZSU2"
                 ,"POLICY_DEVIATION"
                 ,"ZD14"]:
    df_hana = df_hana.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
# Filtering From Hana Table Rows with Positive Quantity 
df_hana = df_hana.filter(df_hana.QUANTITY_TON > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
# Correcting Practiced Price Field
df_hana=df_hana.withColumn("PRACTICED_PRICE_I", when((df_hana.SALES_ORG_COD.like("BRCG")), 
                                                   col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")+col("BX41"))\
                                              .otherwise(col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")+col("BX41")))\
               .withColumn("PRACTICED_PRICE_I", col("PRACTICED_PRICE_I")/col("QUANTITY_TON"))\
               .withColumn("PRACTICED_PRICE_I", col("PRACTICED_PRICE_I")/1000)\
               .withColumn("PRACTICED_PRICE", when((df_hana.SALES_ORG_COD.like("BRCG")), 
                                                   col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")+col("BX41"))\
                                              .otherwise(col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")))\
               .withColumn("PRACTICED_PRICE", col("PRACTICED_PRICE_I")/col("QUANTITY_TON"))\
               .withColumn("PRACTICED_PRICE", col("PRACTICED_PRICE_I")/1000)\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
# Correcting Practiced Price Field
df_hana=df_hana.withColumn("CUSTOMER_GROUP", when(df_hana.CUSTOMER_GROUP.isNull()|
                                                  df_hana.CUSTOMER_GROUP.like("GRUPO DE RENTABILIDADE GENÉRICO")
                                                  ,col("CUSTOMER_DESC")).otherwise(col("CUSTOMER_GROUP")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
# Creating PO Field
df_hana=df_hana.withColumn("PO", when((col("ZD13") < 0)|
                                      (col("ZD14") < 0), col("RBV")
                                                        -abs(col("ZSU2")) 
                                                        +abs(col("ZD13"))     
                                                        +abs(col("ZD06"))
                                                        +abs(col("ZD14")))\
                                       .otherwise(col("RBV")
                                                  -abs(col("ZSU2")) 
                                                  -abs(col("ZEAF")) 
                                                  -abs(col("ZEB1")) 
                                                  +abs(col("ZD13"))     
                                                  +abs(col("ZD06"))
                                                  +abs(col("ZD14"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
#Merge hana and alcada tables
df_hana=df_hana.join(df_alcada.select(df_alcada.SALES_ORG_COD.alias("SALES_ORG_COD_drop")
                                     ,df_alcada.MATERIAL_COD.alias("MATERIAL_COD_drop")
                                     ,df_alcada.ISSUING_STATE.alias("ISSUING_STATE_drop")
                                     ,(df_alcada.VERGE/100).alias("NEW_VERGE"))
                     ,(trim(df_hana.SALES_ORG_COD) == trim(col("SALES_ORG_COD_drop"))) &
                     (df_hana.MATERIAL_COD.cast("int") == col("MATERIAL_COD_drop").cast("int")) &
                     (trim(df_hana.ISSUING_STATE) == trim(col("ISSUING_STATE_drop"))), how="left")\
                .drop("SALES_ORG_COD_drop"
                      ,"MATERIAL_COD_drop"
                      ,"ISSUING_STATE_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
df_hana = df_hana.fillna({'NEW_VERGE':0.04})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
# alcada treatment
df_hana = df_hana.withColumn("VERGE", when(df_hana.VERGE == 0,
                                            col("PO")*col("NEW_VERGE")
                                           ).otherwise(col("VERGE")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
df_hana = df_hana.drop("NEW_VERGE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
# Getting BRANCH Field
df_hana = df_hana.join(df_region2, (replace_some_hifen(replace_some_space(replace_some_apostrophe(df_hana["ISSUING_CITY"]))) == replace_some_hifen(replace_some_space(replace_some_apostrophe(df_region2["CITY_DROP"]))))&
                                   (trim(df_hana["ISSUING_STATE"]) == trim(df_region2["STATE_drop"])), how='left')\
                 .drop("CITY_DROP")\
                 .drop("STATE_drop")\
                 .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, SALES_ORG_COD: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACTICED_PRICE: double, ZD05: float, ZEAF: float, BX41: float, RBV: float, RLV: float, ALCADA: double, ZD01: float, ZD02: float, ZD03: float, ZD04: float, ZD06: float, ZD13: float, PRECO_LISTA: float, ZEB1: float, ZEB2: float, ZSU2: float, DESVIO_POL: float, DESVIO_POL_REAIS: double, ZD14: float, PRACTICED_PRICE_I: double, PO: float, BRANCH: string]

In [118]:
df_hana = df_hana.withColumn("BRANCH", when(df_hana.BRANCH.isNull(), lit("Filial Não Encontrada")).otherwise(df_hana.BRANCH))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
# Filtering 6 Months of Time Horizon
df_hana=df_calendar.join(df_hana, df_calendar.CURRENT_DATE==df_hana.BILLING_DATE, how="inner").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[NUMERIC_DATE: string, CURRENT_DATE: string, CURRENT_WEEK_NUMBER: string, CURRENT_MONTH: string, CURRENT_WEEK_NUMBER_MONTH: string, CURRENT_WEEKEND: string, LMONTH_DATE: string, LMONTH_WEEK_NUMBER: string, LAST_MONTH: string, LMONTH_WEEK_NUMBER_MONTH: string, LMONTH_WEEKEND: string, L2MONTHS_DATE: string, L2MONTHS_WEEK_NUMBER: string, LAST_2_MONTHS: string, L2MONTHS_WEEK_NUMBER_MONTH: string, L2MONTHS_WEEKEND: string, LYEAR_DATE: string, LYEAR_WEEK_NUMBER: string, LYEAR_MONTH: string, LYEAR_WEEK_NUMBER_MONTH: string, LYEAR_WEEKEND: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, SALES_ORG_COD: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACT

In [121]:
df_hana=df_hana.drop("NUMERIC_DATE","CURRENT_DATE","CURRENT_WEEK_NUMBER"
                    ,"CURRENT_MONTH","CURRENT_WEEK_NUMBER_MONTH","CURRENT_WEEKEND"
                    ,"LMONTH_DATE","LMONTH_WEEK_NUMBER","LAST_MONTH","LMONTH_WEEK_NUMBER_MONTH"
                    ,"LMONTH_WEEKEND","L2MONTHS_DATE","L2MONTHS_WEEK_NUMBER","LAST_2_MONTHS"
                    ,"L2MONTHS_WEEK_NUMBER_MONTH","L2MONTHS_WEEKEND","LYEAR_DATE","LYEAR_WEEK_NUMBER"
                    ,"LYEAR_MONTH","LYEAR_WEEK_NUMBER_MONTH","LYEAR_WEEKEND")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
# Putting ZD01, ZD13 and ZD14 on Absoluty 
df_hana = df_hana.withColumn("ZD06",lit(abs(col("ZD06"))))\
                 .withColumn("ZD13",lit(abs(col("ZD13"))))\
                 .withColumn("ZD14",lit(abs(col("ZD14"))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
df_hana = df_hana.drop("ZD05","ZEAF","BX41","ZD01","ZD02"
                       ,"ZD03","ZD04","RPP","ZEB1","ZEB2"
                       ,"ZSU2","LIST_PRICE","PERC_ZD13"
                       ,"PERC_ZD06","FATOR_ZD06","FATOR_ZD13")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
df_hana = df_hana.drop("ZD05")\
                 .drop("ZEAF")\
                 .drop("BX41")\
                 .drop("ZD01")\
                 .drop("ZD02")\
                 .drop("ZD03")\
                 .drop("ZD04")\
                 .drop("ZEB1")\
                 .drop("ZEB2")\
                 .drop("ZSU2")\
                 .drop("LIST_PRICE")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
df_hana = df_hana.withColumn("YEAR_MONTH", concat(year(df_hana.SALES_ORDER_DATE)
                                                      ,when(month(df_hana.SALES_ORDER_DATE) >= "10", month(df_hana.SALES_ORDER_DATE))
                                                                         .otherwise(concat(lit("0"), month(df_hana.SALES_ORDER_DATE)))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----------

# Creating PV

In [126]:
df_pv=df_pv.withColumnRenamed("GPD_COD", "GPD_COD")\
           .withColumnRenamed("DESC_GPD", "GPD_DESC")\
           .withColumnRenamed("DESC_GPM", "GPM_DESC")\
           .withColumnRenamed("SALES_OFFICE", "SALES_OFFICE_COD")\
           .withColumn("YEAR", substring(col("MONTH_YEAR"),1,4))\
           .withColumn("MONTH",substring(col("MONTH_YEAR"),5,6))\
           .withColumnRenamed("VOLUME", "PV_BUDGET")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
df_pv = df_pv.withColumn("SALES_ORG_COD", when(df_pv.SALES_ORG_COD.like("BRCO"), "BRCC")\
                                              .otherwise(when(df_pv.SALES_ORG_COD.like("BRGO"), "BRCG")\
                                                        .otherwise(when(df_pv.SALES_ORG_COD.like("BRIO"), "BRIN")\
                                                                  .otherwise(when(df_pv.SALES_ORG_COD.like("BRDO"), "BRDI")
                                                                            .otherwise(df_pv.SALES_ORG_COD)))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
df_pv = df_pv.filter((df_pv.MONTH == month_ref) & (df_pv.YEAR == year_ref))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
df_pv = df_pv.withColumn("PV_BUDGET", translate(col("PV_BUDGET")," ","").cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
df_pv_new_filter=df_pv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
df_pv_new_filter = df_pv_new_filter.join(df_region3,(upper(trim(df_pv_new_filter["STATE"])) == upper(trim(df_region3["IBGE_UF_ACRONYMS"]))), how='inner')\
                                 .drop("IBGE_UF_ACRONYMS")\
                                 .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
df_pv = df_pv.groupby("SALES_ORG_COD", "STATE", "GPD_COD", "GPD_DESC","GPM_DESC")\
             .sum("PV_BUDGET")\
             .withColumnRenamed("sum(PV_BUDGET)", "PV_BUDGET")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
df_pv = df_pv.join(df_region3,(upper(trim(df_pv["STATE"])) == upper(trim(df_region3["IBGE_UF_ACRONYMS"]))), how='inner')\
                 .drop("IBGE_UF_ACRONYMS")\
                 .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
df_pv.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, STATE: string, GPD_COD: string, GPD_DESC: string, GPM_DESC: string, PV_BUDGET: double, BRANCH: string]

In [135]:
df_pv=df_pv.withColumn("KEY", concat(col("SALES_ORG_COD"),
                                     col("STATE"),
                                     col("BRANCH"),
                                     col("GPD_COD"),
                                     col("GPM_DESC")))\
          .withColumn("KEY_ST", concat(col("SALES_ORG_COD"),
                                     col("STATE"),
                                     col("GPD_COD"),
                                     col("GPM_DESC")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
df_pv_new=df_pv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Creating PV

# 1 Scenery and 2 Scenery

In [137]:
# Filtering 6 Months of Time Horizon
df_hana_6Months=df_hana

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
df_hana_6Months.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, SALES_ORG_COD: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACTICED_PRICE: double, RBV: float, RLV: float, ALCADA: double, ZD06: float, ZD13: float, DESVIO_POL: float, DESVIO_POL_REAIS: double, ZD14: float, PRACTICED_PRICE_I: double, PO: float, BRANCH: string, YEAR_MONTH: string]

In [139]:
df_hana_6Months=df_hana_6Months.drop("NUMERIC_DATE","CURRENT_DATE","CURRENT_WEEK_NUMBER"
                                    ,"CURRENT_MONTH","CURRENT_WEEK_NUMBER_MONTH","CURRENT_WEEKEND"
                                    ,"LMONTH_DATE","LMONTH_WEEK_NUMBER","LAST_MONTH","LMONTH_WEEK_NUMBER_MONTH"
                                    ,"LMONTH_WEEKEND","L2MONTHS_DATE","L2MONTHS_WEEK_NUMBER","LAST_2_MONTHS"
                                    ,"L2MONTHS_WEEK_NUMBER_MONTH","L2MONTHS_WEEKEND","LYEAR_DATE","LYEAR_WEEK_NUMBER"
                                    ,"LYEAR_MONTH","LYEAR_WEEK_NUMBER_MONTH","LYEAR_WEEKEND","YEAR_MONTH", "RLV")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
df_hana_Filial_NDA=df_hana_6Months.filter(df_hana_6Months.BRANCH.like('Filial Não Encontrada'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
df_hana_6Months = df_hana_6Months.filter(~df_hana_6Months.BRANCH.like('Filial Não Encontrada'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
df_hana_6Months_branch=df_hana_6Months.groupby("SALES_ORG_COD", "GPD_COD", "ISSUING_STATE", "BRANCH", "GPD_DESC", "GPM_DESC")\
                                     .sum("QUANTITY_TON")\
                                     .withColumnRenamed("sum(QUANTITY_TON)", "QUANTITY_TON_BRC")\
                                     .withColumnRenamed("SALES_ORG_COD", "SALES_ORG_COD_drop")\
                                     .withColumnRenamed("GPD_COD", "GPD_COD_drop")\
                                     .withColumnRenamed("ISSUING_STATE", "ISSUING_STATE_drop")\
                                     .withColumnRenamed("GPM_DESC", "GPM_DESC_drop")\
                                     .withColumn("KEY", concat(col("SALES_ORG_COD_drop")
                                                               ,col("GPD_COD_drop")
                                                               ,col("ISSUING_STATE_drop")
                                                               ,col("GPM_DESC_drop")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
df_hana_6Months_state=df_hana_6Months.groupby("SALES_ORG_COD", "GPD_COD", "ISSUING_STATE", "GPM_DESC")\
                                     .sum("QUANTITY_TON")\
                                     .withColumnRenamed("sum(QUANTITY_TON)", "QUANTITY_TON_ST")\
                                     .withColumn("KEY_drop", concat(col("SALES_ORG_COD")
                                                                    ,col("GPD_COD")
                                                                    ,col("ISSUING_STATE")
                                                                    ,col("GPM_DESC")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
df_hana_6Months_branch=df_hana_6Months_branch.join(df_hana_6Months_state,df_hana_6Months_state.KEY_drop==df_hana_6Months_branch.KEY,how="inner")\
                                             .drop("KEY_drop"
                                                   ,"KEY"
                                                   ,"SALES_ORG_COD_drop"
                                                   ,"GPD_COD_drop"
                                                   ,"ISSUING_STATE_drop"
                                                   ,"GPM_DESC_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
df_hana_6Months_branch.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[BRANCH: string, GPD_DESC: string, QUANTITY_TON_BRC: double, SALES_ORG_COD: string, GPD_COD: string, ISSUING_STATE: string, GPM_DESC: string, QUANTITY_TON_ST: double]

In [146]:
df_hana_6Months_branch=df_hana_6Months_branch.withColumn("PERC_VOL",col("QUANTITY_TON_BRC")/col("QUANTITY_TON_ST"))\
                                             .withColumn("KEY", concat(col("SALES_ORG_COD")
                                                                       ,col("ISSUING_STATE")
                                                                       ,col("BRANCH")
                                                                       ,col("GPD_COD")
                                                                       ,col("GPM_DESC")
                                                                      ))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
df_hana_6Months_branch_ready=df_hana_6Months_branch.join(df_pv.select("KEY"
                                                                      ,"PV_BUDGET"),on="KEY", how='left_outer')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
df_hana_6Months_branch_ready.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[KEY: string, BRANCH: string, GPD_DESC: string, QUANTITY_TON_BRC: double, SALES_ORG_COD: string, GPD_COD: string, ISSUING_STATE: string, GPM_DESC: string, QUANTITY_TON_ST: double, PERC_VOL: double, PV_BUDGET: double]

In [149]:
df_hana_6Months_branch_ready=df_hana_6Months_branch_ready.fillna(0, subset=['PV_BUDGET'
                                                                            ,'QUANTITY_TON_BRC'
                                                                            ,'QUANTITY_TON_ST'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
df_pv_ready = df_pv.join(df_hana_6Months_branch.select("KEY"
                                                       ,"QUANTITY_TON_BRC"
                                                       ,"QUANTITY_TON_ST"
                                                       ,"PERC_VOL"
                                                       ,"ISSUING_STATE"), on="KEY", how='left_outer')\
                   .drop("STATE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
df_pv_ready.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[KEY: string, SALES_ORG_COD: string, GPD_COD: string, GPD_DESC: string, GPM_DESC: string, PV_BUDGET: double, BRANCH: string, KEY_ST: string, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double, PERC_VOL: double, ISSUING_STATE: string]

In [152]:
df_pv_ready = df_pv_ready.select(col("SALES_ORG_COD")
                                 ,col("ISSUING_STATE")
                                 ,col("GPD_COD")
                                 ,col("GPD_DESC")
                                 ,col("GPM_DESC")
                                 ,col("BRANCH")
                                 ,col("PV_BUDGET")
                                 ,col("QUANTITY_TON_BRC")
                                 ,col("QUANTITY_TON_ST")
                                 ,col("PERC_VOL"))\
                        .filter(df_pv_ready.QUANTITY_TON_BRC.isNull())\
                        .withColumn("PV_BUDGET",lit(0))\
                        .fillna({'PV_BUDGET':0
                                 ,'PERC_VOL':0
                                 ,'QUANTITY_TON_BRC':0
                                 ,"QUANTITY_TON_ST":0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.select("SALES_ORG_COD"
                                                                   ,"GPD_COD"
                                                                   ,"GPD_DESC"
                                                                   ,"GPM_DESC"
                                                                   ,"ISSUING_STATE"
                                                                   ,"BRANCH"
                                                                   ,"QUANTITY_TON_BRC"
                                                                   ,"QUANTITY_TON_ST"
                                                                   ,"PERC_VOL"
                                                                   ,"PV_BUDGET")\
                                                            .unionByName(df_pv_ready)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.withColumn("PV_DONE",col("PV_BUDGET") * col("PERC_VOL"))\
                                                           .fillna({'PV_BUDGET':0
                                                                    ,'PERC_VOL':0
                                                                    ,'QUANTITY_TON_BRC':0
                                                                    ,"QUANTITY_TON_ST":0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
var_year_month=str(year_ref)+str(month_ref).zfill(2)
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.withColumn("YEAR_MONTH", lit(var_year_month))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3 Scenery

In [156]:
df_hana_6Months_state=df_hana_6Months_state.withColumn("KEY_ST",concat(col("SALES_ORG_COD"),col("ISSUING_STATE"),col("GPD_COD"), col("GPM_DESC")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
df_pv_state = df_pv_new.join(df_hana_6Months_state,"KEY_ST",how='left_anti')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
df_pv_state.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[KEY_ST: string, SALES_ORG_COD: string, STATE: string, GPD_COD: string, GPD_DESC: string, GPM_DESC: string, PV_BUDGET: double, BRANCH: string, KEY: string]

In [159]:
df_pv_excential = df_pv_state.select("KEY_ST","GPD_COD","GPD_DESC","GPM_DESC","SALES_ORG_COD","STATE","PV_BUDGET").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
df_branch_state_agg = df_region3.groupby("IBGE_UF_ACRONYMS").agg(countDistinct("BRANCH")\
                                                                .alias("BRANCH_PER_STATE"))\
                                .withColumnRenamed("IBGE_UF_ACRONYMS", "ISSUING_STATE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
df_pv_branch = df_pv_excential.join(df_branch_state_agg, df_pv_excential.STATE == df_branch_state_agg.ISSUING_STATE, how="inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
df_pv_branch.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[KEY_ST: string, GPD_COD: string, GPD_DESC: string, GPM_DESC: string, SALES_ORG_COD: string, STATE: string, PV_BUDGET: double, ISSUING_STATE: string, BRANCH_PER_STATE: bigint]

In [163]:
df_pv_branch = df_pv_branch.withColumn("PV_DONE", col("PV_BUDGET")/col("BRANCH_PER_STATE"))\
                           .drop("BRANCH_PER_STATE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
df_pv_branch = df_pv_branch.select(col("GPD_COD")
                                   ,col("GPD_DESC")
                                   ,col("GPM_DESC")
                                   ,col("SALES_ORG_COD")
                                   ,col("ISSUING_STATE")
                                   ,col("PV_BUDGET")
                                   ,col("PV_DONE"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
df_pv_branch =  df_pv_branch.join(df_region3, upper(trim(df_pv_branch.ISSUING_STATE))==upper(trim(df_region3.IBGE_UF_ACRONYMS)),how='inner')\
                            .withColumn("KEY",concat(col("SALES_ORG_COD")
                                                     ,col("ISSUING_STATE")
                                                     ,col("GPD_COD")
                                                     ,col("BRANCH")
                                                     ,col("GPM_DESC")
                                                    ))\
                            .drop("IBGE_UF_ACRONYMS")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
df_pv_branch.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[GPD_COD: string, GPD_DESC: string, GPM_DESC: string, SALES_ORG_COD: string, ISSUING_STATE: string, PV_BUDGET: double, PV_DONE: double, BRANCH: string, KEY: string]

In [167]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.withColumn("KEY",concat(col("SALES_ORG_COD")
                                                                                    ,col("ISSUING_STATE")
                                                                                    ,col("GPD_COD")
                                                                                    ,col("BRANCH")
                                                                                    ,col("GPM_DESC")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
df_hana_6Months = df_hana_6Months.withColumn("KEY",concat(col("SALES_ORG_COD")
                                                          ,col("ISSUING_STATE")
                                                          ,col("GPD_COD")
                                                          ,col("BRANCH")
                                                          ,col("GPM_DESC")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
df_pv_branch_ready = df_pv_branch.join(df_hana_6Months,"KEY",how="left_anti")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
df_pv_branch_ready.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[KEY: string, GPD_COD: string, GPD_DESC: string, GPM_DESC: string, SALES_ORG_COD: string, ISSUING_STATE: string, PV_BUDGET: double, PV_DONE: double, BRANCH: string]

In [171]:
df_pv_branch_ready = df_pv_branch_ready.select("SALES_ORG_COD","GPD_COD", "GPD_DESC", "GPM_DESC", "ISSUING_STATE","BRANCH","PV_BUDGET","PV_DONE")\
                                       .withColumn("QUANTITY_TON_BRC",lit(0))\
                                       .withColumn("QUANTITY_TON_ST",lit(0))\
                                       .withColumn("PERC_VOL",lit(0))\
                                       .withColumn("YEAR_MONTH",lit(var_year_month))\
                                       .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.select("GPD_COD"
                                                                   ,"GPD_DESC"
                                                                   ,"GPM_DESC"
                                                                   ,"SALES_ORG_COD"
                                                                   ,"ISSUING_STATE"
                                                                   ,"BRANCH"
                                                                   ,"QUANTITY_TON_BRC"
                                                                   ,"QUANTITY_TON_ST"
                                                                   ,"PERC_VOL"
                                                                   ,"PV_BUDGET"
                                                                   ,"PV_DONE"
                                                                   ,"YEAR_MONTH")\
                                                            .unionByName(df_pv_branch_ready).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
df_hana = df_hana.join(df_hana_6Months_branch_ready, on=["SALES_ORG_COD", "GPD_COD", "ISSUING_STATE", "BRANCH", "YEAR_MONTH", "GPD_DESC", "GPM_DESC"], how="left_outer")\
                 .drop("YEAR_MONTH")\
                 .fillna({'PV_BUDGET':0
                          ,'PERC_VOL':0
                          ,'PV_DONE':0})\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, GPD_COD: string, ISSUING_STATE: string, BRANCH: string, GPD_DESC: string, GPM_DESC: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACTICED_PRICE: double, RBV: float, RLV: float, ALCADA: double, ZD06: float, ZD13: float, DESVIO_POL: float, DESVIO_POL_REAIS: double, ZD14: float, PRACTICED_PRICE_I: double, PO: float, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double]

In [175]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.withColumnRenamed("SALES_ORG_COD","OV_AJUST")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
df_hana_6Months_branch_ready=df_hana_6Months_branch_ready.groupBy("OV_AJUST","GPD_COD","BRANCH","YEAR_MONTH", "ISSUING_STATE", "GPD_DESC","GPM_DESC")\
                                                         .agg(sum("PV_BUDGET"),
                                                              sum("PV_DONE"),
                                                              sum("QUANTITY_TON_BRC"),
                                                              sum("QUANTITY_TON_ST"),
                                                              sum("PERC_VOL"))\
                                                    .withColumnRenamed("sum(PV_DONE)","PV_DONE")\
                                                    .withColumnRenamed("sum(PV_BUDGET)","PV_BUDGET")\
                                                    .withColumnRenamed("sum(QUANTITY_TON_BRC)","QUANTITY_TON_BRC")\
                                                    .withColumnRenamed("sum(QUANTITY_TON_ST)","QUANTITY_TON_ST")\
                                                    .withColumnRenamed("sum(PERC_VOL)","PERC_VOL")\
                                                    .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
df_hana_Filial_NDA=df_hana_Filial_NDA.drop("CUSTOMER_COD")\
                                    .drop("CUSTOMER_DESC")\
                                    .drop("CUSTOMER_GROUP")\
                                    .drop("BILLING_NUMBER")\
                                    .drop("BILLING_ITEM")\
                                    .drop("BILLING_DATE")\
                                    .drop("SALES_ORG_COD")\
                                    .drop("SALES_ORDER_DATE")\
                                    .drop("SALES_GROUP_DESC")\
                                    .drop("ISSUING_CITY")\
                                    .drop("MATERIAL_COD")\
                                    .drop("MATERIAL_DESC")\
                                    .drop("ORDER_TYPE")\
                                    .drop("PRACTICED_PRICE")\
                                    .drop("RBV")\
                                    .drop("VERGE")\
                                    .drop("ZD06")\
                                    .drop("ZD13")\
                                    .drop("POLICY_DEVIATION")\
                                    .drop("POLICY_DEVIATION_REAIS")\
                                    .drop("ZD14")\
                                    .drop("PRACTICED_PRICE_I")\
                                    .drop("PO")\
                                    .drop("QUANTITY_TON")\
                                    .drop("SALES_OFFICE_COD")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df_hana_Filial_NDA=df_hana_Filial_NDA.withColumn("PV_BUDGET", lit(0))\
                                      .withColumn("PV_DONE", lit(0))\
                                      .withColumn("QUANTITY_TON_BRC", lit(0))\
                                      .withColumn("QUANTITY_TON_ST", lit(0))\
                                      .withColumn("PERC_VOL", lit(0))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
df_hana_Filial_NDA = df_hana_Filial_NDA.withColumn("YEAR_MONTH", lit(var_year_month))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.unionByName(df_hana_Filial_NDA)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
### Correcting GPM_DESC with blank values
df_hana_6Months_branch_ready=df_hana_6Months_branch_ready.withColumn("GPM_DESC", when(df_hana_6Months_branch_ready.GPM_DESC.isNull(), lit('NÃO INFORMADO'))\
                                                         .otherwise(df_hana_6Months_branch_ready.GPM_DESC))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [182]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.withColumn("COUNTRY", lit("Brazil"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Creating Volume

## Current Calendar

In [183]:
df_final=df_hana.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [184]:
df_final = df_final.withColumn("MANUFACTURER",lit("Gerdau - Praticado"))\
                   .withColumn("CONFIRMED_ORDER_DATE",lit(""))\
                   .withColumn("CREATED_ORDER_DATE",lit(""))\
                   .withColumn("ISSUING_DATE",lit(""))\
                   .withColumn("ORDER_STATUS",lit("Completed"))\
                   .withColumn("TOTAL_WALLET",lit(0).cast("float"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
df_final=df_final.unionByName(df_wallet)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
df_final = df_final.withColumn("COUNTRY", lit("Brazil"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [187]:
### Correcting GPM_DESC with blank values
df_final=df_final.withColumn("GPM_DESC", when(df_final.GPM_DESC.isNull(), lit('NÃO INFORMADO'))\
                                                         .otherwise(df_final.GPM_DESC))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
df_final = df_final.join(df_tvakt, df_final.ORDER_TYPE==df_tvakt.ORDER_TYPE_COD, how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
df_final.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, GPD_COD: string, ISSUING_STATE: string, BRANCH: string, GPD_DESC: string, GPM_DESC: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACTICED_PRICE: double, RBV: float, RLV: float, ALCADA: double, ZD06: float, ZD13: float, DESVIO_POL: float, DESVIO_POL_REAIS: double, ZD14: float, PRACTICED_PRICE_I: double, PO: float, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, TOTAL_WALLET: float, COUNTRY: string, LANGUAGE: string, ORDER_TYPE_DESC: string, ORDER_TYPE_COD: string]

In [190]:
df_final = df_final.drop("ORDER_TYPE_COD"
                         ,"LANGUAGE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [191]:
df_final.write.partitionBy("GPD_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
df_final = df_final.repartition("GPD_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
df_final.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, GPD_COD: string, ISSUING_STATE: string, BRANCH: string, GPD_DESC: string, GPM_DESC: string, CUSTOMER_COD: string, CUSTOMER_DESC: string, CUSTOMER_GROUP: string, BILLING_NUMBER: string, BILLING_ITEM: string, BILLING_DATE: string, OV_AJUST: string, SALES_ORDER_DATE: string, SALES_GROUP_DESC: string, ISSUING_CITY: string, MATERIAL_COD: string, SALES_OFFICE_COD: string, MATERIAL_DESC: string, ORDER_TYPE: string, QUANTITY_TON: float, PRACTICED_PRICE: double, RBV: float, RLV: float, ALCADA: double, ZD06: float, ZD13: float, DESVIO_POL: float, DESVIO_POL_REAIS: double, ZD14: float, PRACTICED_PRICE_I: double, PO: float, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double, PERC_VOL: double, PV_BUDGET: double, PV_DONE: double, FABRICANTE: string, CONFIRMED_ORDER_DATE: string, CREATED_ORDER_DATE: string, ISSUING_DATE: string, ORDER_STATUS: string, TOTAL_WALLET: float, COUNTRY: string, ORDER_TYPE_DESC: string]

-------------

# Creating Filter table

In [194]:
df_pv_new_filter=df_pv_new_filter.withColumn("COUNTRY", lit("Brazil"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
df_filter= df_final.select(df_final.SALES_ORG_COD.alias("OV_AJUST")
                           ,df_final.ISSUING_STATE
                           ,df_final.BRANCH
                           ,df_final.GPD_COD
                           ,df_final.COUNTRY
                           ,df_final.GPD_DESC
                           ,df_final.GPM_DESC
                           ,df_final.SALES_OFFICE_COD).unionByName(df_pv_new_filter.select(df_pv_new_filter.SALES_ORG_COD.alias("OV_AJUST")
                                                         ,df_pv_new_filter.STATE.alias("ISSUING_STATE")
                                                         ,df_pv_new_filter.BRANCH
                                                         ,df_pv_new_filter.GPD_COD
                                                         ,df_pv_new_filter.GPD_DESC
                                                         ,df_pv_new_filter.COUNTRY
                                                         ,df_pv_new_filter.GPM_DESC
                                                         ,df_pv_new_filter.SALES_OFFICE_COD)).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [196]:
df_pex=df_pex.withColumn("COUNTRY", lit("Brazil"))\
             .withColumn("BRANCH", lit(""))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [197]:
df_filter=df_filter.unionByName(df_pex.select(df_pex.SALES_ORG_COD.alias("OV_AJUST")
                                              ,df_pex.STATE.alias("ISSUING_STATE")
                                              ,df_pex.BRANCH
                                              ,df_pex.GPD_COD
                                              ,df_pex.DESC_GPD.alias("GPD_DESC")
                                              ,df_pex.COUNTRY
                                              ,df_pex.DESC_GPM.alias("GPM_DESC")
                                              ,df_pex.SALES_OFFICE.alias("SALES_OFFICE_COD"))).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [198]:
df_filter.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[OV_AJUST: string, ISSUING_STATE: string, BRANCH: string, GPD_COD: string, COUNTRY: string, GPD_DESC: string, GPM_DESC: string, SALES_OFFICE_COD: string]

# Uploading Table

In [199]:
df_filter.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_FILTER, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
df_hana_6Months_branch_ready.write.partitionBy("GPD_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
df_hana_6Months_branch_ready = df_hana_6Months_branch_ready.repartition("GPD_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [202]:
df_hana_6Months_branch_ready.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[OV_AJUST: string, GPD_COD: string, BRANCH: string, YEAR_MONTH: string, ISSUING_STATE: string, GPD_DESC: string, GPM_DESC: string, PV_BUDGET: double, PV_DONE: double, QUANTITY_TON_BRC: double, QUANTITY_TON_ST: double, PERC_VOL: double, COUNTRY: string]

In [203]:
df_hana_6Months_branch_ready.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_SALES_PREV_HEAD, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
# Volume
df_final.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_VOLUME, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…